In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torchaudio
import pandas as pd

from torch import nn
from torch.utils.data import Dataset, DataLoader


from pitch_tracker.utils import dataset
from pitch_tracker.utils.constants import (F_MIN, HOP_LENGTH, N_FFT, N_MELS,
                                           PICKING_FRAME_SIZE,
                                           PICKING_FRAME_STEP,
                                           PICKING_FRAME_TIME, SAMPLE_RATE,
                                           STEP_FRAME, STEP_TIME, WIN_LENGTH,
                                           N_CLASS, )
from pitch_tracker.utils import files

/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/medleydb/__init__.py:69: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  INST_TAXONOMY = yaml.load(fhandle)
/Users/tien.d/workspace/GITHUB/mono_pitch_tracker/medleydb/medleydb/__init__.py:77: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  MIXING_COEFFICIENTS = yaml.load(fhandle)


In [3]:
device = "cuda" if torch.cuda.is_available() \
    else "mps" if torch.backends.mps.is_available() \
    else "cpu"
print(f"Using {device} device")

Using mps device


In [7]:
import os
import pandas as pd

from pitch_tracker.utils import files, dataset
from sklearn.model_selection import train_test_split


DATASET_DIR = '../content/pickled_database/'
SPLIT_BY_DIR = True


if SPLIT_BY_DIR:
    dataset_paths = list(files.list_folder_paths_in_dir(DATASET_DIR))
    train_set, validation_set = train_test_split(dataset_paths, test_size=0.40, random_state=1, shuffle=True)
    validation_set, test_set = train_test_split(validation_set, test_size=0.50, random_state=1, shuffle=True)
    print(f'train_song_set: {len(train_set)}')
    print(f'validation_song_set: {len(validation_set)}')
    print(f'test_song_set: {len(test_set)}')
else:
    dataset_paths = files.list_all_file_paths_in_dir(DATASET_DIR)
    train_set, validation_set = train_test_split(dataset_paths, test_size=0.40, random_state=1, shuffle=True)
    validation_set, test_set = train_test_split(validation_set, test_size=0.50, random_state=1, shuffle=True)
    print(f'train_set: {len(train_set)}')
    print(f'validation_set: {len(validation_set)}')
    print(f'test_set: {len(test_set)}')

train_song_set: 64
validation_song_set: 21
test_song_set: 22


In [9]:
train_dataset = dataset.AudioDataset(train_set)
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
for i_batch, (feature, label) in enumerate(train_dataloader):
    print(i_batch, feature.shape, len(label))
    


0 torch.Size([8, 1050, 88]) 3
1 torch.Size([8, 1050, 88]) 3
2 torch.Size([8, 1050, 88]) 3
3 torch.Size([8, 1050, 88]) 3
4 torch.Size([8, 1050, 88]) 3
5 torch.Size([8, 1050, 88]) 3
6 torch.Size([8, 1050, 88]) 3
7 torch.Size([8, 1050, 88]) 3
8 torch.Size([8, 1050, 88]) 3
9 torch.Size([8, 1050, 88]) 3
10 torch.Size([8, 1050, 88]) 3
11 torch.Size([8, 1050, 88]) 3
12 torch.Size([8, 1050, 88]) 3
13 torch.Size([8, 1050, 88]) 3
14 torch.Size([8, 1050, 88]) 3
15 torch.Size([8, 1050, 88]) 3
16 torch.Size([8, 1050, 88]) 3
17 torch.Size([8, 1050, 88]) 3
18 torch.Size([8, 1050, 88]) 3
19 torch.Size([8, 1050, 88]) 3
20 torch.Size([8, 1050, 88]) 3
21 torch.Size([8, 1050, 88]) 3
22 torch.Size([8, 1050, 88]) 3
23 torch.Size([8, 1050, 88]) 3
24 torch.Size([8, 1050, 88]) 3
25 torch.Size([8, 1050, 88]) 3
26 torch.Size([8, 1050, 88]) 3
27 torch.Size([8, 1050, 88]) 3
28 torch.Size([8, 1050, 88]) 3
29 torch.Size([8, 1050, 88]) 3
30 torch.Size([8, 1050, 88]) 3
31 torch.Size([8, 1050, 88]) 3
32 torch.Size([8, 

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Conv2d(1, 32, 3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3),
            # nn.Linear(6*, 512),
            # nn.ReLU(),
            # nn.Linear(512, 10),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  )
)


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (feature, (_, duration, pitch)) in enumerate(dataloader):
        feature = feature.to(device)
        duration = duration.to(device)
        pitch = pitch.to(device)

        # Compute prediction error
        pred = model(feature)
        loss = loss_fn(pred, (duration, pitch))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(feature)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")            

In [ ]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test_loop(validation_dataloader, model, loss_fn)
print("Done!")

In [ ]:
label_folder = '../content/gen_label/2560/Melody2_midi/'
label_dict = dataset.create_label_dict_from_dir(label_folder)
dataset_paths = dataset.create_dataset_path_dict(label_folder)
feature_label_gen = dataset.create_feature_label_generator(
    dataset_path_dict=dataset_paths, 
    sample_rate= SAMPLE_RATE,
    n_fft= N_FFT,
    n_mels= N_MELS,
    n_class= N_CLASS,
    hop_length= HOP_LENGTH,
    picking_frame_step= PICKING_FRAME_STEP,
    picking_frame_size= PICKING_FRAME_SIZE,
    step_frame= STEP_FRAME,
    step_time= STEP_TIME,
    dist_threshold= 0.1,
    empty_threshold= 0.3,
)

for song_name, feature_label_pair in feature_label_gen:
    n_big_frame = 0
    for feature, label in feature_label_pair:
        print(feature.shape)
        print(label[0].shape)
        print(label[1].shape)
        print(label[2].shape)
        n_big_frame+=1
        break
    break

In [ ]:
feature_torch = torch.from_numpy(feature).unsqueeze(dim=0)
feature_torch.shape

In [ ]:
learning_rate = 1e-3
batch_size = 8
epochs = 5

conv2d_layer_1 = nn.Conv2d(1, 32, 3)
batchnorm2d_layer_1 = nn.BatchNorm2d(32)
relu_layer_1 = nn.ReLU()
conv2d_layer_2 = nn.Conv2d(32, 32, 3)

out = conv2d_layer_1(feature_torch.type(torch.float32))
out = batchnorm2d_layer_1(out)
out = relu_layer_1(out)
out = conv2d_layer_2(out)
out.shape

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
out1_np=out1.type(torch.int).numpy()
n_channels, w, h = out1_np.shape
nn_channels = int((n_channels+1)/2)
fig, axes = plt.subplots(nn_channels, nn_channels, figsize=(8,8))
for i in range(nn_channels):
    for j in range(nn_channels):
        axes[i,j].imshow(out1_np[i+j].T)
# plt.imshow(out1_np[2])

In [ ]:
nn_channels